### Task 1. 

#### MapReduce

Use MapReduce to calculate the average housing price in each zip code and filter out records in which gross sqft is less than 500 and sale price is less than 100000. Round the average housing price to 2 decimals, and sort the output by housing price ascending.

In [ ]:
!wget -O housingSalseSample.csv https://github.com/CUSP2022ADS/Data/raw/main/housingSalseSample.csv

In [ ]:
# code here
import functools
from functools import reduce
import csv



def returnInfo(record): #extract necessary fields from a data record (raw)
    return(record['ZIP CODE'],record['GROSS SQUARE FEET'],record['SALE PRICE'])

def filterData(record): #filter the records to keep only those listings over 500 sqfeet and 100000 dollars
    # filter function must return True or False
    if record[1]=='':
        area = 0.0
    if record[1]!='':
        area = float(record[1])
    if record[2]=='':
        price = 0.0
    if record[2]!='':
        price = float(record[2])
    if area > 500 and price > 100000:
        return True
    else:
        return False
    
def TotalPrice(result,record): #find the total price and number of properties by zipcode
    zipcde,saleprice = record[0],float(record[2])

    if zipcde not in result.keys():
        result[zipcde] = [saleprice,1]
    if zipcde in result.keys():
        old_count = result[zipcde][1]
        oldprice = result[zipcde][0]
        new_count = old_count+1
        new_price = ((oldprice+saleprice))
        result.update({zipcde:[new_price,new_count]})

    return result


with open('housingSalseSample.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    output = list(map(lambda x: (x[0],round((x[1][0]/x[1][1]), 2)), #find the average price dividing the price of all houses by number of houses
                      reduce(TotalPrice,
                             filter(filterData, 
                                    map(returnInfo,reader)),{}).items()))



In [ ]:
output = sorted(output, key=lambda x: x[1])

In [ ]:
'''grading function
please pass your output into this function and upload the generated csv file 
together with your notebook to your GitHub repo'''

def grading(output):
    import pandas as pd
    pd.DataFrame(output,columns=['zipcode','price']).to_csv('output1.csv',index=False)

grading(output)

### Task 2. 
#### multiprocessing and Chunk

In [ ]:
!wget -O ADSSession2Task2Data.csv https://raw.githubusercontent.com/CUSP2022ADS/Data/main/ADSSession2Task2Data.csv

Apply the chunk and MapReduce method to sum up each type's value in the synthetic dataset—only keep records which $Select$ value is 1, and read 1000 lines in each chunk. Use multiprocessing package to deploy the task to multi-cores. The output should only have two columns: type and total value.

Hint: filter function is not available in multiprocessing, implement an if-else function in the map or reduce step.

In [1]:
import functools
from functools import reduce
import csv
from multiprocessing import Pool


def returnInfo1(record): #extract necessary fields from a data record (raw)
    return(record['Type'],record['Value'],record['Select'])

def SumValues(result,record):
    typez,vals,sel = record[0],record[1],record[2]
    

    if(sel=='1'):
        
        if typez not in result.keys():
            result[typez] = float(vals)
            
        else:
            old_val = result[typez]
            new_val = 0
            
            if isinstance(old_val, list):
              for i in old_val:
                new_val = i+float(vals)
            else:
              new_val = old_val+float(vals)
            result.update({typez:new_val})
            
    return result

100% [........................................................] 279369 / 279369

In [2]:
with open('ADSSession2Task2Data.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    chunk = []
    result1 = []
    chunksize = 1000
    pool = Pool(8)
    for i, line in enumerate(reader):
        if (i % chunksize == 0 and i > 0):
            output = list(reduce(SumValues,pool.map(returnInfo1,chunk),{}).items())
            result1 += output
            chunk = []
        chunk.append(line)
    output = list(reduce(SumValues,pool.map(returnInfo1,chunk),{}).items())
    result1 += output
    pool.close() 

In [3]:
'''grading function
please pass your output into this function and upload the generated csv file 
together with your notebook to your GitHub repo'''

def grading(output):
    import pandas as pd
    pd.DataFrame(output,columns=['Type','Value']).to_csv('output2.csv',index=False)

grading(result1) #Returns sum chunk by chunk

In [ ]:
def returnInfo2(record): #extract necessary fields from a data record (raw)
    return(record['Type'],record['Value'])

def SumValues2(result,record):
    typez,vals = record[0],record[1]
    

    if typez not in result.keys():
        result[typez] = float(vals)
        
    else:
        old_val = result[typez]
        new_val = 0
        
        if isinstance(old_val, list):
          for i in old_val:
            new_val = i+float(vals)
        else:
          new_val = old_val+float(vals)
          result.update({typez:new_val})
          
    return result

with open('output2.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    output3 = list(reduce(SumValues2,map(returnInfo2,reader),{}).items())

In [ ]:
def grading(output):
    import pandas as pd
    pd.DataFrame(output,columns=['Type','Value']).to_csv('output3.csv',index=False)

grading(output3) #Returns sum over the entire dataset